In [1]:
import pandas as pd

In [2]:
words_path = '/media/alex/nvme-a/vosk-model-ru-0.10/graph/words.txt'
words = pd.read_csv(words_path, sep=" ")
words.columns = ['line', 'id']
words[words.line.str.contains('шолохово')]

,line,id


### Форматирование и очистка примеров

In [59]:
def word_data(path):
    
    with open(path, 'rt', encoding="utf8", errors='ignore') as file:
        words = file.read()
    words = words.lower()
    
    correct = '\nйцукенгшщзхъфывапролджэячсмитьбю '
    used = ''.join(list(set(words)))
    for u in used:
        if u not in correct:
            words = words.replace(u, ' ')
    while '  ' in words:
        words = words.replace('  ', ' ')
    words = words.replace(' ', '\n')
    
    with open(path.replace('.csv','_b.csv'), 'w') as file:
        file.write(words)
        
    words = pd.read_csv(path.replace('.csv','_b.csv'), header=None)
    words.columns = ['word']    
    words.drop_duplicates(inplace=True)
    words = words.dropna()
    
    return words

In [60]:
def get_missing(words_path):
    df = pd.DataFrame(columns = ['word'])
    for path in words_path:
        df = pd.concat([df, word_data(path)], axis = 0)
    df.drop_duplicates(inplace=True)
    df = df.dropna()
    df = pd.DataFrame(df[df.word.str.len()>3])
    df.to_csv('csv/all_words.csv', index = False)
    missing_words = df[df.word.isin(words.line)==False]
    missing_words.to_csv('csv/missing_words.csv', index = False)
    print('слов в модели:', len(words))
    print('слов в 1С:', len(df))
    print('слов, отсутствующих в модели:', len(missing_words))    
    return missing_words

### Отсутсвующие слова

In [61]:
# Улицы
words_path = ['csv/streets_msk.csv', 'csv/streets_spb.csv', 'csv/streets_reg.csv']
missing_words = get_missing(words_path)

слов в модели: 709156
слов в 1С: 29155
слов, отсутствующих в модели: 15724


In [62]:
# Неисправности
words_path = ['csv/neispr_msk.csv', 'csv/neispr_spb.csv', 'csv/neispr_reg.csv']
missing_words = get_missing(words_path)

слов в модели: 709156
слов в 1С: 1016
слов, отсутствующих в модели: 40


In [64]:
# Бренды
words_path = ['csv/brands_msk.csv', 'csv/brands_spb.csv', 'csv/brands_reg.csv']
missing_words = get_missing(words_path)

слов в модели: 709156
слов в 1С: 81
слов, отсутствующих в модели: 24
